## Как оценить качество генерации?

Задача работы - адаптировать llm под онлайн-разговорную лексику. => метрика должна учитывать функциональный стиль сообщений. Для этого можно обучить нейронную сеть на отдельном датасете разговорной лексики. Например, собрать его из открытых бесед. Эта модель будет оценивать вероятности сгенерированного текста основной модели по перплексии (либо попробовать сразу с перплексией основной модели).


В то же время необходимо оценить адекватность сгенерированного комментария. В качестве такой меры можно оценить семантическую близость сгенерированного комментария с живыми комментариями. Например, через bert_score. Справедливо брать самый высокий результат - это покажет, что смысл сгенерированного комментария совпадает с одним из живых => модель адекватно реагирует на пост и улавливает семантику

Результатом могут служить как две разные метрики, так и их взвешенная совокупность подобно F-$\beta$ мере

# BERT_score

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# !pip install -q bert_score
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from bert_score import score as bertscore_score
import numpy as np

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_dir = "/content/drive/MyDrive/diploma/weights_model"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir).to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50262, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50262, bias=False)
)

In [14]:
gen_kwargs = dict(
    max_new_tokens=40,
    do_sample=True,
    top_k=50,
    top_p=0.9,
    temperature=0.8,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

def generate_comment(post_text, cluster_id, num_variants):
  prompt = f"Пост: {post_text} Тип реакции: <cluster_{cluster_id}>\nКомментарий:"
  inputs = tokenizer(prompt, return_tensors="pt") #.to(device)

  outputs = model.generate(**inputs, **gen_kwargs, num_return_sequences=num_variants)
  results = []
  for out in outputs:
      text = tokenizer.decode(out, skip_special_tokens=True)
      # берём только сам комментарий
      if "Комментарий:" in text:
          comment = text.split("Комментарий:")[-1].strip()
      else:
          # дебаг на случай, если комментарий пропадет
          comment = text[len(prompt):].strip()
      results.append(comment)
  return results

def best_bertscore_vs_candidates(gen_text, candidate_texts, lang="ru"):
    # bertscore_score возвращает (P - precision, R - recall, F - f1-мера) торч-тензоры
    P, R, F = bertscore_score([gen_text]*len(candidate_texts), candidate_texts, lang=lang, verbose=False)
    F_np = F.cpu().numpy()
    P_np = P.cpu().numpy()
    R_np = R.cpu().numpy()
    idx = int(F_np.argmax())
    return float(F_np[idx]), float(P_np[idx]), float(R_np[idx])


В собранном датасете большая доля данных на данный момент с новостных пабликов петербурга @piterach и @mashmoyka, найдем случайный паблик похожей тематики и оценим качество на одном из постов. В дальнейшем оценка будет происходить по отдельной тестовой выборке.
Пост: https://t.me/lifepetersburg/47249

In [15]:
dataset_list = [
    {"post":"""🦷 Через пять лет люди смогут отращивать новые зубы
Исследователи разработали препарат — он запускает рост зубов в любом возрасте. Средство подавляет белок USAG-1, мешающий регенерации, и способствует формированию крепких зубов с эмалью и нервами. Первые тесты на людях уже идут, побочных эффектов пока не обнаружено.
Если испытания завершатся успешно — к 2030 году средство может появиться в клиниках по всему миру""",
     "top_comments": [
         "А до этого, как  пять лет ходить то без зубов?))",
         "Ооо супер😂😂",
         "интересно, сколько это будет стоить. 😟",
         "Да ну не, опять зубные боли, кариес... лучше протезы",
         "Бред какой-то. С новыми зубами и опухоли начнут расти по всему телу. 🙈"
     ],
     "pred_cluster": 3  # выбрал случайно, так нужно проанализировать как-то?
    },
]

scores = []
p_scores = []
r_scores = []

for item in dataset_list:
    post = item["post"]
    top5 = item["top_comments"][:5]
    cluster = item["pred_cluster"]
    # генерируем n_variants комментариев
    gen_list = generate_comment(post, cluster, num_variants=3)
    for gen in gen_list:
      best_f1, best_p, best_r = best_bertscore_vs_candidates(gen, top5, lang="ru")

      scores.append(best_f1)
      p_scores.append(best_p)
      r_scores.append(best_r)
      print(scores, p_scores, r_scores)

# Итог по всему датасету
print("Mean and median best BERTScores F1:", np.mean(scores), "\t", np.median(scores))
print("Mean best P:", np.mean(p_scores), "Mean best R:", np.mean(r_scores))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

[0.7215716242790222] [0.7035408020019531] [0.7405511140823364]
[0.7215716242790222, 0.6565693020820618] [0.7035408020019531, 0.6462758779525757] [0.7405511140823364, 0.6671959161758423]
[0.7215716242790222, 0.6565693020820618, 0.67633455991745] [0.7035408020019531, 0.6462758779525757, 0.6579878330230713] [0.7405511140823364, 0.6671959161758423, 0.6957337856292725]
Mean and median best BERTScores F1: 0.6848251620928446 	 0.67633455991745
Mean best P: 0.6692681709925333 Mean best R: 0.7011602719624838


Результат вроде неплохой, но если открыть эти комментарии, то, несмотря на их сходство по словам к живым, адекватность хромает. => можно оценивать по семантической близости n-грам (тогда "зубные щетки начал делать" должно получать меньшую близость, чем по раздельности). возможно, сказывается малость модели и датасета, тогда при использовании модели по типу яндекс гпт (которая должна лучше улавливать грамматику русского) количество бреда уменьшится сам по себе

В текущей модели 125М параметров https://arxiv.org/pdf/2309.10931

В яндекс гпт - 8 Млрд  https://huggingface.co/yandex/YandexGPT-5-Lite-8B-instruct

In [16]:
gen_list

['Ну ты же видишь как у всех шикарные ровненькие зубы) Но это еще пол беды если их нет))',
 'Но я его только ради одного съел да еще чистить стал регулярно ((( как можно сохранить здоровые зубки если их нет',
 'Я вот сейчас зубные щетки только начал делать. Чуть что сразу вспоминаю о наращивании когтеточки😂']